In [100]:
import pandas as pd
import numpy as np

# usersDF = pd.read_json("./data/processed/converted/users.json")
# sessionsDF = pd.read_json("./data/processed/converted/sessions.json")
# productsDF = pd.read_json("./data/processed/converted/products.json")
# deliveriesDF = pd.read_json("./data/processed/converted/deliveries.json")

usersDF = pd.read_json("./data/raw/users.jsonl", lines=True)
sessionsDF = pd.read_json("./data/raw/sessions.jsonl", lines=True)
productsDF = pd.read_json("./data/raw/products.jsonl", lines=True)
deliveriesDF = pd.read_json("./data/raw/deliveries.jsonl", lines=True)


Searching for anomalies and missing data.

In [101]:
usersNan = usersDF.isna().sum()
print(f"NaN values in user data\n{usersNan}\n")

productsNan = productsDF.isna().sum()
print(f"NaN values in products data\n{productsNan}\n")

sessionsNan = sessionsDF.isna().sum()
print(f"NaN values in sessions data\n{sessionsNan}\n")

NaN values in user data
user_id    0
name       0
city       0
street     0
dtype: int64

NaN values in products data
product_id       0
product_name     0
category_path    0
price            0
user_rating      0
dtype: int64

NaN values in sessions data
session_id               0
timestamp                0
user_id                  0
product_id               0
event_type               0
offered_discount         0
purchase_id         114253
dtype: int64



In [102]:
mergedDF = pd.merge(sessionsDF, usersDF, on="user_id")
mergedDF = pd.merge(mergedDF, productsDF, on="product_id")

mergedNan = mergedDF.isna().sum()
print(f"NaN values in merged data\n{mergedNan}\n")

NaN values in merged data
session_id               0
timestamp                0
user_id                  0
product_id               0
event_type               0
offered_discount         0
purchase_id         114253
name                     0
city                     0
street                   0
product_name             0
category_path            0
price                    0
user_rating              0
dtype: int64



In [177]:
noNanSessionsDF = sessionsDF.dropna()
noNanSessionsDF.event_type.value_counts() == noNanSessionsDF.event_type.count()
noNanSessionsDF.count()/sessionsDF.count()


session_id          0.087042
timestamp           0.087042
user_id             0.087042
product_id          0.087042
event_type          0.087042
offered_discount    0.087042
purchase_id         1.000000
dtype: float64

In [104]:
statsDF = mergedDF.groupby(pd.Grouper(key="timestamp", axis=0, freq="1D", sort=True))
productsViewStats = statsDF["product_id"].count().describe()
print(f"Products view stats\n{productsViewStats}")

productsBuyStats = statsDF["purchase_id"].count().describe()
print(f"Purchase stats\n{productsBuyStats}")



Products view stats
count    300.000000
mean     417.153333
std       62.835320
min      233.000000
25%      372.000000
50%      414.500000
75%      456.250000
max      632.000000
Name: product_id, dtype: float64
Purchase stats
count    300.000000
mean      36.310000
std        6.030072
min       23.000000
25%       32.000000
50%       36.000000
75%       40.000000
max       59.000000
Name: purchase_id, dtype: float64


In [105]:
usersDF.street

0             al. Kamienna 87
1           ul. Szpitalna 638
2              ulica Dolna 26
3         ul. Osiedlowa 23/03
4       aleja Krakowska 84/76
                ...          
195           ul. Pogodna 975
196          plac Rolna 73/65
197       plac Piaskowa 83/06
198    plac Storczykowa 45/59
199          plac Zielona 336
Name: street, Length: 200, dtype: object

In [129]:
productsStatsDF = productsDF.groupby(["category_path"])
display(productsStatsDF.price.describe())
productsDF.loc[productsDF["price"] == productsDF["price"].min()]
productsDF.loc[productsDF["price"] < 1]


,count,mean,std,min,25%,50%,75%,max
category_path,,,,,,,,
Gry i konsole;Gry komputerowe,202.0,37.995248,32.529106,1.00,18.4050,31.945,39.9900,199.90
Gry i konsole;Gry na konsole;Gry PlayStation3,9.0,87.630000,22.287582,59.90,79.9000,79.900,99.9900,129.00
Gry i konsole;Gry na konsole;Gry Xbox 360,32.0,76.674375,28.442363,14.99,49.9900,74.945,89.9900,139.99
Komputery;Drukarki i skanery;Biurowe urządzenia wielofunkcyjne,9.0,4238.504444,1872.985317,1998.14,2399.0000,4598.000,5259.0000,7639.00
Komputery;Monitory;Monitory LCD,17.0,955.805294,693.800589,269.00,609.0000,739.000,1079.0000,3029.00
Komputery;Tablety i akcesoria;Tablety,2.0,2066.995000,353.588746,1816.97,1941.9825,2066.995,2192.0075,2317.02
Sprzęt RTV;Audio;Słuchawki,1.0,553.000000,NaN,553.00,553.0000,553.000,553.0000,553.00
Sprzęt RTV;Przenośne audio i video;Odtwarzacze mp3 i mp4,2.0,71.850000,9.970206,64.80,68.3250,71.850,75.3750,78.90
Sprzęt RTV;Video;Odtwarzacze DVD,2.0,151.000000,59.396970,109.00,130.0000,151.000,172.0000,193.00


,product_id,product_name,category_path,price,user_rating


In [153]:
productsDF.describe()



,product_id,price,user_rating
count,319.000000,319.000000,319.000000
mean,1160.000000,247.787962,2.648154
std,92.231593,813.788548,1.488174
min,1001.000000,1.000000,0.013905
25%,1080.500000,24.990000,1.342497
50%,1160.000000,41.000000,2.780512
75%,1239.500000,92.995000,3.932414
max,1319.000000,7639.000000,4.993596
